# Hyperparameter Tuning
Find the most acceptable parameter

In [1]:
from optimization.acs import ACS_VRP,ACS_TSP
from optimization.bso import BSO_VRP,BSO_TSP
from optimization.hybrid_acs_bso import ACSBSO_VRP,ACSBSO_TSP

from optimization.aco import ACO_VRP,ACO_TSP
from optimization.haco import HACO_VRP,HACO_TSP

from optimization.koneksi import ConDB

import sql_connection
import json
import time
import random
import datetime
import copy

## Data for Tuning

In [7]:
def generate_data(n = 30, random_state = None):
    random.seed(random_state)
    query = """SELECT 
                    p.post_id,
                    p.post_type,
                    CASE
                        WHEN pj.pj_jam_buka = pj.pj_jam_tutup THEN "tutup"
                        ELSE "buka"
                    END AS is_operate
                FROM 
                    posts p
                LEFT JOIN
                    posts_jadwal pj
                    ON p.post_id = pj.pj_id_tempat AND pj.pj_hari = "minggu"
                """

    df_location = sql_connection.read_from_sql(query)
    
    tourid = df_location[(df_location['post_type']=="location")&
                         (df_location['is_operate'] != "tutup")]['post_id'].values.tolist()
    tourid = random.sample(tourid,n)
    idhotel = df_location[df_location['post_type']=="hotel"]['post_id'].values.tolist()
    idhotel = idhotel[random.randint(0,len(idhotel)-1)]
    
    db = ConDB()

    hotel = db.HotelbyID(idhotel)
    tur = db.WisatabyID(tourid)
    timematrix = db.TimeMatrixbyID(hotel._id,tourid)
    
    return hotel,tur,timematrix

In [8]:
# setting
hotel,tur,timematrix = generate_data(n=30,random_state=30)
travel_days = 3
dwaktu,dtarif,drating = 1,1,1

C:\Users\rahma\Documents\kuliah\thesis\rahmat_code\travel-itinerary-recommendation\sql_connection.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dataset = pd.read_sql(query, mydb)


In [9]:
db = ConDB()

hotel = db.HotelbyID(idhotel)
tur = db.WisatabyID(tourid)
timematrix = db.TimeMatrixbyID(hotel._id,tourid)

## Maximum Iteration

In [10]:
# List of iterations
iterations = [100, 200, 300]

#list of algorithms
for n in iterations:
    print(f"n = {n}")
    
    print()
    print("ACO - VRP")
    start = time.time()
    aco_vrp = ACO_VRP(max_iter = n, alpha=1,beta=4,q0=0.6,init_pheromone=0.1,rho=0.7,num_ant=35,random_state=100)
    aco_vrp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = aco_vrp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    print("==============================================")
    
    print()
    print("ACO - TSP")
    start = time.time()
    aco_tsp = ACO_TSP(max_iter = n,alpha=1,beta=4,q0=0.6,init_pheromone=0.1,rho=0.7,num_ant=35,random_state=100)
    aco_tsp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = aco_tsp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    print("==============================================")
    
    print()
    print("HACO - VRP")
    start = time.time()
    haco_vrp = HACO_VRP(max_iter = n,alpha=1,beta=4,q0=0.6,init_pheromone=0.1,rho=0.7,num_ant=35,q1=0.6,q2=0.3,q3=0.6,random_state=100)
    haco_vrp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = haco_vrp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    print("==============================================")
    
    print()
    print("HACO - TSP")
    start = time.time()
    haco_tsp = HACO_TSP(max_iter = n,alpha=1,beta=4,q0=0.6,init_pheromone=0.1,rho=0.7,num_ant=35,q1=0.6,q2=0.3,q3=0.6,random_state=100)
    haco_tsp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = haco_tsp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    print("==============================================")

n = 100

ACO - VRP
fitness = 0.6265744382533959
time = 6.22437047958374

ACO - TSP
fitness = 0.5870652609344841
time = 7.1122658252716064

HACO - VRP
fitness = 0.6332510743289997
time = 9.349596977233887

HACO - TSP
fitness = 0.5768651971072661
time = 12.46596646308899
n = 200

ACO - VRP
fitness = 0.6265744382533959
time = 6.216338634490967

ACO - TSP
fitness = 0.5870652609344841
time = 7.6476757526397705

HACO - VRP
fitness = 0.6332510743289997
time = 9.369270324707031

HACO - TSP
fitness = 0.5768651971072661
time = 12.28934359550476
n = 300

ACO - VRP
fitness = 0.6265744382533959
time = 6.099036455154419

ACO - TSP
fitness = 0.5870652609344841
time = 7.224016189575195

HACO - VRP
fitness = 0.6332510743289997
time = 9.585718154907227

HACO - TSP
fitness = 0.5768651971072661
time = 12.47077488899231


**results:**
- ACO-based bisa menggunakan max iter = 100

## Tuning ACO dan HACO (VRP/TSP)
Tuning dilakukan dengan VRP, TSP mengikuti

default parameter/parameter awal:
- alpha = 1
- beta = 4
- q0 = 0.6
- init_pheromone = 0.1
- rho = 0.7
- num_ant = 35

### Find Init Pheromone

In [11]:
#find init pheromone
init_pheromone = [0.1,0.3,0.5,1]

for pheromone in init_pheromone:
    print(f"pheromone = {pheromone}")
    
    print("ACO - VRP")
    start = time.time()
    aco_vrp = ACO_VRP(max_iter = 100, alpha=1,beta=4,q0=0.6,init_pheromone=pheromone,rho=0.7,num_ant=35,random_state=100)
    aco_vrp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = aco_vrp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    
    print("-----------------------------------------------")
    
    print("ACO - TSP")
    start = time.time()
    aco_tsp = ACO_TSP(max_iter = 100, alpha=1,beta=4,q0=0.6,init_pheromone=pheromone,rho=0.7,num_ant=35,random_state=100)
    aco_tsp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = aco_tsp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    
    print("-----------------------------------------------")
    
    print("HACO - VRP")
    start = time.time()
    haco_vrp = HACO_VRP(max_iter = 100,alpha=1,beta=4,q0=0.6,init_pheromone=pheromone,rho=0.7,num_ant=35,q1=0.6,q2=0.3,q3=0.6,random_state=100)
    haco_vrp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = haco_vrp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    
    print("-----------------------------------------------")
    
    print("HACO - TSP")
    start = time.time()
    haco_tsp = HACO_TSP(max_iter = 100,alpha=1,beta=4,q0=0.6,init_pheromone=pheromone,rho=0.7,num_ant=35,q1=0.6,q2=0.3,q3=0.6,random_state=100)
    haco_tsp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = haco_tsp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")

    print("==============================================")

pheromone = 0.1
ACO - VRP
fitness = 0.6265744382533959
time = 6.3339269161224365
-----------------------------------------------
ACO - TSP
fitness = 0.5870652609344841
time = 7.357358455657959
-----------------------------------------------
HACO - VRP
fitness = 0.6332510743289997
time = 9.618029594421387
-----------------------------------------------
HACO - TSP
fitness = 0.5768651971072661
time = 12.622501850128174
pheromone = 0.3
ACO - VRP
fitness = 0.6347900506209255
time = 5.948105812072754
-----------------------------------------------
ACO - TSP
fitness = 0.5776809288393605
time = 6.943788766860962
-----------------------------------------------
HACO - VRP
fitness = 0.6332510743289997
time = 9.355708122253418
-----------------------------------------------
HACO - TSP
fitness = 0.5773321372917709
time = 12.759697675704956
pheromone = 0.5
ACO - VRP
fitness = 0.6265867556605925
time = 6.051199913024902
-----------------------------------------------
ACO - TSP
fitness = 0.50233681285

### Find Alpha

In [12]:
#find alpha_t
alpha_ls = [1,2,3,4,5]

for alpha in alpha_ls:
    print(f"alpha_t = {alpha}")
    
    print("ACO - VRP")
    start = time.time()
    aco_vrp = ACO_VRP(max_iter = 100, alpha=alpha,beta=4,q0=0.6,init_pheromone=0.3,rho=0.7,num_ant=35,random_state=100)
    aco_vrp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = aco_vrp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    
    print("-----------------------------------------------")
    
    print("ACO - TSP")
    start = time.time()
    aco_tsp = ACO_TSP(max_iter = 100, alpha=alpha,beta=4,q0=0.6,init_pheromone=0.1,rho=0.7,num_ant=35,random_state=100)
    aco_tsp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = aco_tsp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    
    print("-----------------------------------------------")
    
    print("HACO - VRP")
    start = time.time()
    haco_vrp = HACO_VRP(max_iter = 100,alpha=alpha,beta=4,q0=0.6,init_pheromone=0.5,rho=0.7,num_ant=35,q1=0.6,q2=0.3,q3=0.6,random_state=100)
    haco_vrp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = haco_vrp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    
    print("-----------------------------------------------")
    
    print("HACO - TSP")
    start = time.time()
    haco_tsp = HACO_TSP(max_iter = 100,alpha=alpha,beta=4,q0=0.6,init_pheromone=0.5,rho=0.7,num_ant=35,q1=0.6,q2=0.3,q3=0.6,random_state=100)
    haco_tsp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = haco_tsp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")

    print("==============================================")

alpha_t = 1
ACO - VRP
fitness = 0.6347900506209255
time = 5.84489893913269
-----------------------------------------------
ACO - TSP
fitness = 0.5870652609344841
time = 7.135408878326416
-----------------------------------------------
HACO - VRP
fitness = 0.6372228643278687
time = 17.295751810073853
-----------------------------------------------
HACO - TSP
fitness = 0.5894339872953442
time = 11.756479263305664
alpha_t = 2
ACO - VRP
fitness = 0.6256557743506584
time = 5.907141208648682
-----------------------------------------------
ACO - TSP
fitness = 0.5290630496162193
time = 7.224557876586914
-----------------------------------------------
HACO - VRP
fitness = 0.6274586006149429
time = 17.689531326293945
-----------------------------------------------
HACO - TSP
fitness = 0.5834841083644323
time = 11.672152280807495
alpha_t = 3
ACO - VRP
fitness = 0.6322477537939681
time = 6.234119415283203
-----------------------------------------------
ACO - TSP
fitness = 0.4985916502785972
time =

### Find beta

In [13]:
#find beta
beta_ls = [1,2,3,4,5]

for beta in beta_ls:
    print(f"beta = {beta}")
    
    print("ACO - VRP")
    start = time.time()
    aco_vrp = ACO_VRP(max_iter = 100, alpha=1,beta=beta,q0=0.6,init_pheromone=0.3,rho=0.7,num_ant=35,random_state=100)
    aco_vrp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = aco_vrp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    
    print("-----------------------------------------------")
    
    print("ACO - TSP")
    start = time.time()
    aco_tsp = ACO_TSP(max_iter = 100, alpha=5,beta=beta,q0=0.6,init_pheromone=0.1,rho=0.7,num_ant=35,random_state=100)
    aco_tsp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = aco_tsp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    
    print("-----------------------------------------------")
    
    print("HACO - VRP")
    start = time.time()
    haco_vrp = HACO_VRP(max_iter = 100,alpha=1,beta=beta,q0=0.6,init_pheromone=0.5,rho=0.7,num_ant=35,q1=0.6,q2=0.3,q3=0.6,random_state=100)
    haco_vrp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = haco_vrp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    
    print("-----------------------------------------------")
    
    print("HACO - TSP")
    start = time.time()
    haco_tsp = HACO_TSP(max_iter = 100,alpha=1,beta=beta,q0=0.6,init_pheromone=0.5,rho=0.7,num_ant=35,q1=0.6,q2=0.3,q3=0.6,random_state=100)
    haco_tsp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = haco_tsp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    
    print("==============================================")

beta = 1
ACO - VRP
fitness = 0.6275598879888458
time = 5.890435218811035
-----------------------------------------------
ACO - TSP
fitness = 0.5847001438508322
time = 7.150310277938843
-----------------------------------------------
HACO - VRP
fitness = 0.6359731781929422
time = 9.18321442604065
-----------------------------------------------
HACO - TSP
fitness = 0.5106135511125679
time = 13.485067129135132
beta = 2
ACO - VRP
fitness = 0.6177073330381564
time = 5.990638256072998
-----------------------------------------------
ACO - TSP
fitness = 0.5597641028442405
time = 7.3985276222229
-----------------------------------------------
HACO - VRP
fitness = 0.6333673912876807
time = 13.30143427848816
-----------------------------------------------
HACO - TSP
fitness = 0.5837939482980367
time = 22.0753915309906
beta = 3
ACO - VRP
fitness = 0.6281038093157072
time = 5.630902290344238
-----------------------------------------------
ACO - TSP
fitness = 0.5620466925809006
time = 7.081247329711

### Find q0

In [15]:
#find beta
q0_ls = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]

for q0 in q0_ls:
    print(f"q0 = {q0}")
    
    print("ACO - VRP")
    start = time.time()
    aco_vrp = ACO_VRP(max_iter = 100, alpha=1,beta=4,q0=q0,init_pheromone=0.3,rho=0.7,num_ant=35,random_state=100)
    aco_vrp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = aco_vrp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    
    print("-----------------------------------------------")
    
    print("ACO - TSP")
    start = time.time()
    aco_tsp = ACO_TSP(max_iter = 100, alpha=5,beta=4,q0=q0,init_pheromone=0.1,rho=0.7,num_ant=35,random_state=100)
    aco_tsp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = aco_tsp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    
    print("-----------------------------------------------")
    
    print("HACO - VRP")
    start = time.time()
    haco_vrp = HACO_VRP(max_iter = 100,alpha=1,beta=4,q0=q0,init_pheromone=0.5,rho=0.7,num_ant=35,q1=0.6,q2=0.3,q3=0.6,random_state=100)
    haco_vrp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = haco_vrp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    
    print("-----------------------------------------------")
    
    print("HACO - TSP")
    start = time.time()
    haco_tsp = HACO_TSP(max_iter = 100,alpha=1,beta=3,q0=q0,init_pheromone=0.5,rho=0.7,num_ant=35,q1=0.6,q2=0.3,q3=0.6,random_state=100)
    haco_tsp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = haco_tsp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    
    print("==============================================")

q0 = 0.1
ACO - VRP
fitness = 0.6247498998506863
time = 10.15181851387024
-----------------------------------------------
ACO - TSP
fitness = 0.5480394633817691
time = 7.687492847442627
-----------------------------------------------
HACO - VRP
fitness = 0.641349603044539
time = 16.3848397731781
-----------------------------------------------
HACO - TSP
fitness = 0.518209990349248
time = 22.769992113113403
q0 = 0.2
ACO - VRP
fitness = 0.6329112755120623
time = 9.30800747871399
-----------------------------------------------
ACO - TSP
fitness = 0.5382620452310721
time = 8.010656833648682
-----------------------------------------------
HACO - VRP
fitness = 0.6395885684110859
time = 18.128830194473267
-----------------------------------------------
HACO - TSP
fitness = 0.5717765817362154
time = 23.106305837631226
q0 = 0.3
ACO - VRP
fitness = 0.6259437515453662
time = 5.955664157867432
-----------------------------------------------
ACO - TSP
fitness = 0.5684565143111763
time = 7.7217195034

### Find Rho

In [16]:
#find rho
rho_ls = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]

for rho in rho_ls:
    print(f"rho = {rho}")
    
    print("ACO - VRP")
    start = time.time()
    aco_vrp = ACO_VRP(max_iter = 100, alpha=1,beta=4,q0=0.6,init_pheromone=0.3,rho=rho,num_ant=35,random_state=100)
    aco_vrp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = aco_vrp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    
    print("-----------------------------------------------")
    
    print("ACO - TSP")
    start = time.time()
    aco_tsp = ACO_TSP(max_iter = 100, alpha=5,beta=4,q0=0.8,init_pheromone=0.1,rho=rho,num_ant=35,random_state=100)
    aco_tsp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = aco_tsp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    
    print("-----------------------------------------------")
    
    print("HACO - VRP")
    start = time.time()
    haco_vrp = HACO_VRP(max_iter = 100,alpha=1,beta=4,q0=0.4,init_pheromone=0.5,rho=rho,num_ant=35,q1=0.6,q2=0.3,q3=0.6,random_state=100)
    haco_vrp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = haco_vrp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    
    print("-----------------------------------------------")
    
    print("HACO - TSP")
    start = time.time()
    haco_tsp = HACO_TSP(max_iter = 100,alpha=1,beta=3,q0=0.8,init_pheromone=0.5,rho=rho,num_ant=35,q1=0.6,q2=0.3,q3=0.6,random_state=100)
    haco_tsp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = haco_tsp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    
    print("==============================================")

rho = 0.1
ACO - VRP
fitness = 0.6434329363778725
time = 6.360431432723999
-----------------------------------------------
ACO - TSP
fitness = 0.6198007548917087
time = 13.405211448669434
-----------------------------------------------
HACO - VRP
fitness = 0.6403748603180627
time = 15.76816439628601
-----------------------------------------------
HACO - TSP
fitness = 0.5970298540790959
time = 13.502090692520142
rho = 0.2
ACO - VRP
fitness = 0.6312564186605484
time = 5.813015460968018
-----------------------------------------------
ACO - TSP
fitness = 0.6198007548917087
time = 13.03552770614624
-----------------------------------------------
HACO - VRP
fitness = 0.6361940577435065
time = 11.680840730667114
-----------------------------------------------
HACO - TSP
fitness = 0.6198007548917087
time = 13.806185007095337
rho = 0.3
ACO - VRP
fitness = 0.6390911905022033
time = 6.2515318393707275
-----------------------------------------------
ACO - TSP
fitness = 0.6198007548917087
time = 13.

### Find num_ant

In [17]:
#find num_ant
num_ant_ls = [10,30,35,50]

for num_ant in num_ant_ls:
    print(f"num_ant = {num_ant}")
    
    print("ACO - VRP")
    start = time.time()
    aco_vrp = ACO_VRP(max_iter = 100, alpha=1,beta=4,q0=0.6,init_pheromone=0.3,rho=0.1,num_ant=num_ant,random_state=100)
    aco_vrp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = aco_vrp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    
    print("-----------------------------------------------")
    
    print("ACO - TSP")
    start = time.time()
    aco_tsp = ACO_TSP(max_iter = 100, alpha=5,beta=4,q0=0.8,init_pheromone=0.1,rho=0.7,num_ant=num_ant,random_state=100)
    aco_tsp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = aco_tsp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    
    print("-----------------------------------------------")
    
    print("HACO - VRP")
    start = time.time()
    haco_vrp = HACO_VRP(max_iter = 100,alpha=1,beta=4,q0=0.4,init_pheromone=0.5,rho=0.7,num_ant=num_ant,q1=0.6,q2=0.3,q3=0.6,random_state=100)
    haco_vrp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = haco_vrp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    
    print("-----------------------------------------------")
    
    print("HACO - TSP")
    start = time.time()
    haco_tsp = HACO_TSP(max_iter = 100,alpha=1,beta=3,q0=0.8,init_pheromone=0.5,rho=0.7,num_ant=num_ant,q1=0.6,q2=0.3,q3=0.6,random_state=100)
    haco_tsp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = haco_tsp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    
    print("==============================================")

num_ant = 10
ACO - VRP
fitness = 0.6317642667249355
time = 3.268063545227051
-----------------------------------------------
ACO - TSP
fitness = 0.6179944644743072
time = 4.033811092376709
-----------------------------------------------
HACO - VRP
fitness = 0.6336857487047676
time = 5.203694581985474
-----------------------------------------------
HACO - TSP
fitness = 0.6180459048035253
time = 3.5296263694763184
num_ant = 30
ACO - VRP
fitness = 0.632247753793968
time = 4.917611837387085
-----------------------------------------------
ACO - TSP
fitness = 0.6198007548917087
time = 5.910403728485107
-----------------------------------------------
HACO - VRP
fitness = 0.6445580683928768
time = 15.1826491355896
-----------------------------------------------
HACO - TSP
fitness = 0.6009174717630963
time = 9.950869798660278
num_ant = 35
ACO - VRP
fitness = 0.6434329363778725
time = 6.447542190551758
-----------------------------------------------
ACO - TSP
fitness = 0.6198007548917087
time = 

## Tuning HACO (VRP/TSP)
Tuning parameter dari ACO dan mutasinya

default parameter:
- q1 = 0.6
- q2 = 0.3
- q3 = 0.6

parameter beta = 1

### Find q1

In [19]:
#find q1
q1_ls = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]

for q1 in q1_ls:
    print(f"q1 = {q1}")
    
    print("HACO - VRP")
    start = time.time()
    haco_vrp = HACO_VRP(max_iter = 100,alpha=1,beta=4,q0=0.4,init_pheromone=0.5,rho=0.7,num_ant=30,q1=q1,q2=0.3,q3=0.6,random_state=100)
    haco_vrp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = haco_vrp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    
    print("-----------------------------------------------")
    
    print("HACO - TSP")
    start = time.time()
    haco_tsp = HACO_TSP(max_iter = 100,alpha=1,beta=3,q0=0.8,init_pheromone=0.5,rho=0.7,num_ant=35,q1=q1,q2=0.3,q3=0.6,random_state=100)
    haco_tsp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = haco_tsp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    
    print("==============================================")

q1 = 0.1
HACO - VRP
fitness = 0.6305039786590918
time = 7.281928300857544
-----------------------------------------------
HACO - TSP
fitness = 0.6198007548917087
time = 8.008650302886963
q1 = 0.2
HACO - VRP
fitness = 0.6287108052004807
time = 6.6494152545928955
-----------------------------------------------
HACO - TSP
fitness = 0.5366546487490441
time = 8.5749351978302
q1 = 0.3
HACO - VRP
fitness = 0.6295008922393388
time = 13.321787118911743
-----------------------------------------------
HACO - TSP
fitness = 0.6198007548917087
time = 9.848313331604004
q1 = 0.4
HACO - VRP
fitness = 0.6382681452347136
time = 14.266581535339355
-----------------------------------------------
HACO - TSP
fitness = 0.6212410841098157
time = 9.634897232055664
q1 = 0.5
HACO - VRP
fitness = 0.6269463587818012
time = 9.391258478164673
-----------------------------------------------
HACO - TSP
fitness = 0.6198007548917087
time = 12.06723403930664
q1 = 0.6
HACO - VRP
fitness = 0.6445580683928768
time = 15.52933

### Find q2

In [21]:
#find q2
q2_ls = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]

for q2 in q2_ls:
    print(f"q2 = {q2}")
    
    print("HACO - VRP")
    start = time.time()
    haco_vrp = HACO_VRP(max_iter = 100,alpha=1,beta=4,q0=0.4,init_pheromone=0.5,rho=0.7,num_ant=30,q1=0.6,q2=q2,q3=0.6,random_state=100)
    haco_vrp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = haco_vrp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    
    print("-----------------------------------------------")
    
    print("HACO - TSP")
    start = time.time()
    haco_tsp = HACO_TSP(max_iter = 100,alpha=1,beta=3,q0=0.8,init_pheromone=0.5,rho=0.7,num_ant=35,q1=0.4,q2=q2,q3=0.6,random_state=100)
    haco_tsp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = haco_tsp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    
    print("==============================================")

q2 = 0.1
HACO - VRP
fitness = 0.6364790520220577
time = 9.309593677520752
-----------------------------------------------
HACO - TSP
fitness = 0.4828081184675334
time = 10.353280544281006
q2 = 0.2
HACO - VRP
fitness = 0.6285636144606026
time = 16.79152011871338
-----------------------------------------------
HACO - TSP
fitness = 0.6198007548917087
time = 10.089799165725708
q2 = 0.3
HACO - VRP
fitness = 0.6445580683928768
time = 15.747766256332397
-----------------------------------------------
HACO - TSP
fitness = 0.6212410841098157
time = 10.435264825820923
q2 = 0.4
HACO - VRP
fitness = 0.6433468990130741
time = 16.32557511329651
-----------------------------------------------
HACO - TSP
fitness = 0.6198007548917087
time = 10.012335538864136
q2 = 0.5
HACO - VRP
fitness = 0.6317310161944795
time = 9.660943984985352
-----------------------------------------------
HACO - TSP
fitness = 0.5830788630321571
time = 9.707260608673096
q2 = 0.6
HACO - VRP
fitness = 0.6319650660985469
time = 14.3

### Find q3

In [22]:
#find q3
q3_ls = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]

for q3 in q3_ls:
    print(f"q3 = {q3}")
    
    print("HACO - VRP")
    start = time.time()
    haco_vrp = HACO_VRP(max_iter = 100,alpha=1,beta=4,q0=0.4,init_pheromone=0.5,rho=0.7,num_ant=30,q1=0.6,q2=0.3,q3=q3,random_state=100)
    haco_vrp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = haco_vrp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    
    print("-----------------------------------------------")
    
    print("HACO - TSP")
    start = time.time()
    haco_tsp = HACO_TSP(max_iter = 100,alpha=1,beta=3,q0=0.8,init_pheromone=0.5,rho=0.7,num_ant=35,q1=0.4,q2=0.3,q3=q3,random_state=100)
    haco_tsp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = haco_tsp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    
    print("==============================================")

q3 = 0.1
HACO - VRP
fitness = 0.6343877235150588
time = 11.555824756622314
-----------------------------------------------
HACO - TSP
fitness = 0.6198007548917087
time = 10.523557186126709
q3 = 0.2
HACO - VRP
fitness = 0.641406993803749
time = 16.08614683151245
-----------------------------------------------
HACO - TSP
fitness = 0.6198007548917087
time = 9.810739755630493
q3 = 0.3
HACO - VRP
fitness = 0.6415088859754543
time = 10.179952621459961
-----------------------------------------------
HACO - TSP
fitness = 0.6198007548917087
time = 10.073296546936035
q3 = 0.4
HACO - VRP
fitness = 0.6410941767726429
time = 10.897185802459717
-----------------------------------------------
HACO - TSP
fitness = 0.6180459048035253
time = 10.897355794906616
q3 = 0.5
HACO - VRP
fitness = 0.6387311081976764
time = 9.950395584106445
-----------------------------------------------
HACO - TSP
fitness = 0.6180459048035253
time = 10.149662733078003
q3 = 0.6
HACO - VRP
fitness = 0.6445580683928768
time = 15.

### Check TSP

In [47]:
start = time.time()
haco_tsp = HACO_TSP(init_pheromone=1,alpha = 1,beta = 1,q0=0.1,q1=0.6,q2=0.3,q3=0.3,rho=0.7,num_ant=35,max_iter = 100,random_state=100)
haco_tsp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
fitness = haco_tsp.construct_solution()[-1]
end = time.time()
print(f"fitness = {fitness}")
print(f"time = {end-start}")

fitness = 0.5752660425783128
time = 25.350791454315186


choosen parameter VRP:
- q1 = 0.9
- q2 = 0.5
- q3 = 0.3

choosen parameter TSP:
- q1 = 0.6
- q2 = 0.3
- q3 = 0.3